In [1]:
import io
import os
import fitz  # PyMuPDF for PDF parsing
import pdf2image
import pytesseract
from PIL import Image
from docx import Document
from paddleocr import PaddleOCR

# Path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# Specific windows paths
#win_poppler_path = "G:\\My Drive\\AI_Lead\\Research_N_Development\\CrewAI\\CrewAI\CrewAI_Research\\crewai_main\\windows_tools\\Release-24.02.0-0.zip"
#win_poppler_path = "G:\\My Drive\\AI_Lead\\Research_N_Development\\CrewAI\\CrewAI\\CrewAI_Research\\rewai_main\\tools\\windows_tools\\poppler-24.02.0\\Library\\bin"


In [23]:
# Process the input documents 
class document_processing():    

    def __init__(self, file_path):
        self.file_path = file_path
        self.status = {'status' : False, 'reason': 'Not processed yet'}      
        self.file_extension = os.path.splitext(self.file_path )[1].lower() # Get the file extension
        self.paddle_obj = PaddleOCR()
        
    def text_preprocessing(self, text):

        """
        Clean and preprocess text.

        Args:
            text (str): The input text to be cleaned.

        Returns:
            str: The cleaned text.

        """
        # Define a dictionary for character replacements
        replacements = {
        #    '-': ' ',
            # '|': '',
            # '[': '',
            # ']': '',
            # '}': '',
            # '{': '',
            # '__': '',
            # '_': '',
            # '——': '',
            # '—': '',
            # "\'": '',
            # "'": '',
            # "”": '',
            # "‘": '',
            # '"': '',
            # "°": '',
            # '=': '',
            # '@': '&',
            # "£": 'E',
            # "€": 'E',
            # "«": '',
            # ":": '',
            # ";": '',
            # "»": '',
            # "¢": '',
            # "#": '',
            # "~": '',
            # "..": '',
            # "“": '',
            # '″': '',
            # "(s)": 's',
            # "“": '',
        }
        
        #Lowercase
        #text = text.lower()

        # Apply replacements
        for old, new in replacements.items():
            text = text.replace(old, new)

        # Clean extra spaces and remove lines
       # text = ' '.join(text.split())
        #text = '\n'.join([line.strip() for line in text.split('\n') if line.strip() != ''])# complete remove new empty lines
        text = '\n'.join([line.strip() for i, line in enumerate(text.split('\n')) if line.strip() != '' or (i > 0 and text.split('\n')[i-1].strip() != '')]) # leave one line space

        text = text.strip()
    
        return text
        
    def paddle_ocr(self, image):
        try:
            if isinstance(image, str):
                result = self.paddle_obj.ocr(image)# det=False
                text = '\n'.join([word[1][0] for line in result for word in line])
            else:
                img_byte_arr = io.BytesIO()
                image.save(img_byte_arr, format='PNG')
                image_bytes = img_byte_arr.getvalue()
                result = self.paddle_obj.ocr(image_bytes)# det=False
                text = '\n'.join([word[1][0] for line in result for word in line])
            text = self.text_preprocessing(text)
        except Exception as e:
            print("Error in paddle OCR: ", e)
            text=''
        return text


    def tesseract_ocr(self, image):
        try:
            if isinstance(image, str):
                # Open the image file
                with Image.open(image) as img:
                    # Perform OCR on the image
                    text = pytesseract.image_to_string(img)
            else:
                text = pytesseract.image_to_string(image)
                
            text = self.text_preprocessing(text)
        except Exception as e:
            print("Error in tesseract OCR: ", e)
            text=''
        return text
        
    def image_ocr(self, image, ocr_type=1):
        if ocr_type == 0:
            text = self.paddle_ocr(image)
            # If paddle fails
            if not text:
                text = self.tesseract_ocr(image)
            return text
        elif ocr_type == 1:
            text = self.tesseract_ocr(image)
            # If tesseract fails
            if not text:
                text = self.paddle_ocr(image)
            return text
        else:
            print("Unsupported OCR type, Please select between paddle(0) and tesseract(1) ")
            return None

    def document_selectable_ocr(self, file_path):
        text = ''
        # OCR - Selectable text
        with fitz.open(file_path) as file:
            for page_num in range(len(file)):
                page = file.load_page(page_num)
                text += page.get_text()
        text = self.text_preprocessing(text)
        return text
    
    def document_scanned_ocr(self, file_path, doc_type):
        text = ''
        if doc_type=='pdf':
            #Convert pdf to image
            pages = pdf2image.convert_from_path(file_path, 300)# set dpi=300, first_page = 1, last_page = 1)
            for index, image in enumerate(pages):
                text += self.image_ocr(image)
                
        elif doc_type=='word':
    
            try:
                # Open the Word document
                doc = Document(file_path)
                # Extract images from the document
                images = []
                for rel in doc.part.rels.values():
                    if "image" in rel.target_ref:
                        image_data = rel.target_part.blob
                        # Convert image data to PIL Image
                        img = Image.open(io.BytesIO(image_data))
                        images.append(img)
                # Perform OCR on each image
                for img in images:
                    text_temp = self.image_ocr(img)
                    text += text_temp + "\n"
                    
            except Exception as e:
                print("Error in processing Word document: ", e)
            
        return text
    

    def main_ocr(self, file_path, doc_type, ocr_type=0):
        
        """
        paddle = 0
        tesseract = 1 
        """
        
        text = ''

        if doc_type == 'image':
            text = self.image_ocr(file_path)
        
        elif doc_type == 'word':
            # Selectable text OCR
            text = self.document_selectable_ocr(file_path)
            
            if len(text)<=10:
                print("No text found, Checking scanned word doc:")   
                text = self.document_scanned_ocr(file_path, doc_type) 
    
        elif doc_type == 'pdf':
            
            # Selectable text OCR
            text = self.document_selectable_ocr(file_path)
            
            if len(text)<=10:
                print("No text found, Checking scanned pdf:")   
                text = self.document_scanned_ocr(file_path, doc_type) 
    
        return text

    def document_ocr(self):

        # Check if the documnet exists
        if not os.path.exists(self.file_path):
            print("Document not found.")
            self.status['reason'] = 'Document not found'
            return self.status
        
        if self.file_extension == '.pdf':
            print("Received PDF file extension!")
            
            text = self.main_ocr(self.file_path, 'pdf') 
     
            if text:
                self.status['status'] = True
                self.status['reason'] = 'PDF document text extracted'
                self.status['text'] = text
                
            else:
                self.status['status'] = False
                self.status['reason'] = 'PDF document extraction failed'
                self.status['text'] = ''
                
            return self.status
        
        
        elif self.file_extension == '.docx':
            print("Received word file extension!")

            text = self.main_ocr(self.file_path, 'word') 

            
            if text:
                self.status['status'] = True
                self.status['reason'] = 'Word document text extracted'
                self.status['text'] = text
                
            else:
                self.status['status'] = False
                self.status['reason'] = 'Word document extraction failed'
                self.status['text'] = ''
                
            return self.status     
                    
            return images_text
        
        elif self.file_extension in ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']:
            print("Received image file extension!")
        
            text = self.main_ocr(self.file_path, 'image') 
     
            if text:
                self.status['status'] = True
                self.status['reason'] = 'Image document text extracted'
                self.status['text'] = text
                
            else:
                self.status['status'] = False
                self.status['reason'] = 'Image document extraction failed'
                self.status['text'] = ''
                
            return self.status
        
        else:
            self.status['status'] = False
            self.status['reason'] = 'Unsupported file format. Supported formats are images, pdf and word documents.'
            self.status['text'] = ''
                
            return self.status
        
        
        return self.status
    
    
    

In [41]:


# Example:
pdf_file_path_selectable = "G:\\My Drive\\AI_Lead\\Research_N_Development\\CrewAI\\CrewAI\\CrewAI_Research\\test_files\\pdf_data_selectable.pdf"
pdf_file_path_scanned = "G:\\My Drive\\AI_Lead\\Research_N_Development\\CrewAI\\CrewAI\\CrewAI_Research\\test_files\\pdf_data_scanned.pdf"

image_file_path = "G:\\My Drive\\AI_Lead\\Research_N_Development\\CrewAI\\CrewAI\\CrewAI_Research\\test_files\\image_data.jpg"
word_file_path_selectable = "G:\\My Drive\\AI_Lead\\Research_N_Development\\CrewAI\\CrewAI\\CrewAI_Research\\test_files\\word_data_selectable.docx"
word_file_path_scanned = "G:\\My Drive\\AI_Lead\\Research_N_Development\\CrewAI\\CrewAI\\CrewAI_Research\\test_files\\word_data_scanned.docx"

doc_processing = document_processing(image_file_path)

[2024/04/26 06:45:04] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\abdul.wajid/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\abdul.wajid/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, 

In [42]:
a =doc_processing.document_ocr()
print(a)

Received image file extension!
{'status': True, 'reason': 'Image document text extracted', 'text': 'Processing Date :\n\nSequence No: 1\n\nSubSeqNo : 0\nPRO IVY MEDICAL GROUP PROFESSIONAL CORPORATION\nFront Image\n\nWorkSource :\n\nPage: 1 Date: 11/30/2022\nCity NATIONAL BANK\nBAN Rec COMPANY\n11/30/2022 Batch Number : 00242849 Transaction No : 1\nPaid Amount : $0.00 Site :\nImageSeq1 : 2 ImageSeq2 : (e)\n\nVERE NEE EE RRR EZ\nSee\n\nCHECK REFERENCE\n\nHelmsman |x: 0084090511\nzz ManagemeneSerdiens LLC SCHECK\'AMOUNT..., ,».._ .| BLOCK NUMBER\nBe vee cellyt 2H | weeeegeayog 2 \'[e" 01142972),\n\nPO BOX 7070 .\nLONDON, KY 40742\n\nSEND ORIGINAL ‘BILLS TO:\n\nHELMSMAN MANAGEMENT SERVICES\n\nPAGE 10F 3\n\nLOS ANGELES LA\n\nPO BOX 7203\nLONDON, KY 40742\n\nOSN: MM03501112511-000301\nBANK: 298\n\nweet CHECK REF: 0084090511 DATE: 11/25/22 AMT: 641.00\nCLAIM NO. WC 608-E80376 HOD INTERNAL BILL NO: 136856556 MSR: N1525548\nCONTRACT NO: WP8- 65B-290306-299 CUST/EXTERNAL BILL NO: © 1115048231\n\

In [43]:
print(a.get('text'))

Processing Date :

Sequence No: 1

SubSeqNo : 0
PRO IVY MEDICAL GROUP PROFESSIONAL CORPORATION
Front Image

WorkSource :

Page: 1 Date: 11/30/2022
City NATIONAL BANK
BAN Rec COMPANY
11/30/2022 Batch Number : 00242849 Transaction No : 1
Paid Amount : $0.00 Site :
ImageSeq1 : 2 ImageSeq2 : (e)

VERE NEE EE RRR EZ
See

CHECK REFERENCE

Helmsman |x: 0084090511
zz ManagemeneSerdiens LLC SCHECK'AMOUNT..., ,».._ .| BLOCK NUMBER
Be vee cellyt 2H | weeeegeayog 2 '[e" 01142972),

PO BOX 7070 .
LONDON, KY 40742

SEND ORIGINAL ‘BILLS TO:

HELMSMAN MANAGEMENT SERVICES

PAGE 10F 3

LOS ANGELES LA

PO BOX 7203
LONDON, KY 40742

OSN: MM03501112511-000301
BANK: 298

weet CHECK REF: 0084090511 DATE: 11/25/22 AMT: 641.00
CLAIM NO. WC 608-E80376 HOD INTERNAL BILL NO: 136856556 MSR: N1525548
CONTRACT NO: WP8- 65B-290306-299 CUST/EXTERNAL BILL NO: © 1115048231

DOCUMENT NO: 11150482310 BR PROVIDER #: 853776300-0002

PAYEE: PRO IVY MEDICAL GROUP PROFESSIO PATIENT ACCT. #: NA
TAX ID: 85-3776300 SSN: XXX-KX-83

In [18]:
import os
import fitz  # PyMuPDF for PDF parsing
from docx import Document
from PIL import Image
from paddleocr import PaddleOCR
from io import BytesIO

    
    
# Process the input documents 
class document_processing():    

    def __init__(self, file_path):
        self.file_path = file_path
        self.status = {'status' : False, 'reason': 'Not processed yet'}

    def document_ocr(self):
        document_text = ''
        
        # Check if the documnet exists
        if not os.path.exists(self.file_path):
            print("Document not found.")
            self.status['reason'] = 'Document not found'
            return self.status
        
        
        
        return(document_text)
            


    def ocr_image(self, image_path):
        # Perform OCR on the image
        ocr = PaddleOCR()
        result = ocr.ocr(image_path, det=False)
        image_text = ""
        for line in result:
            for word in line:
                image_text += word[0] + " "
            image_text += "\n"
        return image_text


    def ocr_file(self, file_path):
        # Check if the file exists
        if not os.path.exists(file_path):
            print("File not found.")
            return
        
        # Get the file extension
        file_extension = os.path.splitext(file_path)[1].lower()

        if file_extension == '.pdf':
            # Perform OCR on PDF
            with fitz.open(file_path) as pdf_file:
                text = ""
                for page_num in range(len(pdf_file)):
                    page = pdf_file.load_page(page_num)
                    text += page.get_text()
                        
            # if len(text)<=10:
            #     print("No text found, Checking scanned pdf:")   
            #     text = paddle_OCR(file_path) 
            #     print("Length: ", length(text)
                    
            return text
        elif file_extension == '.docx':
            # Perform OCR on .docx images
            document = Document(file_path)
            images_dir = os.path.splitext(file_path)[0] + "_images"
            if not os.path.exists(images_dir):
                os.makedirs(images_dir)

            images_text = ""
            images = []
            for idx, shape in enumerate(document.inline_shapes):
                if shape._inline:
                    image_data = shape._inline.graphic.graphicData.pic.blipFill.blip.embed
                    image_part = document.part.related_parts[image_data]
                    image_stream = image_part.blob
                    image = Image.open(BytesIO(image_stream))
                    filename = os.path.join("data", f"image_{idx}.png")
                    image.save(filename)
                    print(filename)
                    image_text = self.ocr_image(filename)
                    images_text += image_text + "\n"
            return images_text
        else:
            print("Unsupported file format. Supported formats are .pdf and .docx.")
            return


In [19]:
# Example usage:
file_path = "G:\My Drive\AI_Lead\Research_N_Development\CrewAI\CrewAI\CrewAI_Research\\test_files\MAILING_2024_04_22_12_59_42_1713772782-PHhZ211713772782_15_opt_0.pdf"
a = document_processing(file_path)
text = a.ocr_file(file_path)
print("+++++++++++>",text)


+++++++++++> 


In [11]:
len(text)

0

In [5]:

import os
import fitz  # PyMuPDF for PDF parsing
from docx import Document
from PIL import Image
import pytesseract
from io import BytesIO

def ocr_image(image_path):
    # Perform OCR on the image using Tesseract OCR
    image_text = pytesseract.image_to_string(Image.open(image_path))
    return image_text


def ocr_file(file_path):
    # Check if the file exists
    if not os.path.exists(file_path):
        print("File not found.")
        return
    
    # Get the file extension
    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.pdf':
        # Perform OCR on PDF
        with fitz.open(file_path) as pdf_file:
            text = ""
            for page_num in range(len(pdf_file)):
                page = pdf_file.load_page(page_num)
                text += page.get_text()
        return text
    elif file_extension == '.docx':
        # Perform OCR on .docx images
        document = Document(file_path)
        images_dir = os.path.splitext(file_path)[0] + "_images"
        if not os.path.exists(images_dir):
            os.makedirs(images_dir)

        images_text = ""
        images = []
        for idx, shape in enumerate(document.inline_shapes):
            if shape._inline:
                image_data = shape._inline.graphic.graphicData.pic.blipFill.blip.embed
                image_part = document.part.related_parts[image_data]
                image_stream = image_part.blob
                image = Image.open(BytesIO(image_stream))
                filename = os.path.join("data", f"image_{idx}.png")
                image.save(filename)
                print(filename)
                image_text = ocr_image(filename)
                images_text += image_text + "\n"
        return images_text
    else:
        print("Unsupported file format. Supported formats are .pdf and .docx.")
        return
